# 자연어 처리 (NLP)
 > 자연어 처리 및 분석이란 텍스트 데이터와 같은 비정형의 자연어를 이해하고 해석하여, 이에 관한 인사이트를 도출해내는 분석 기법을 의미합니다.

## 목차
- import Base code
    - word cloud / 단어 빈도 분석
    - word_relative / 연관어 분석
    - key_sentence / 핵심 문장 추출
- 함수 시연
    - word cloud / 단어 빈도 분석
    - word_relative / 연관어 분석
    - key_sentence / 핵심 문장 추출

## import Base code

In [1]:
! pip install textrank==0.1.2

You should consider upgrading via the '/Users/jheok/Desktop/mobigen/Template-Jupyter-Sample/venv/bin/python -m pip install --upgrade pip' command.


In [2]:
import os
import pandas as pd

from collections import Counter
from konlpy.tag import Mecab
from nltk import sent_tokenize

path = os.path.join(os.getcwd(), "sample_nlp.csv")

res = []
with open(path, "r") as fs:
    for line in fs:
        res.append(line)

lines = " ".join(res)

## word cloud / 단어 빈도 분석
> 단어 빈도 분석이란 특정 문서에서 자주 사용되는 단어를 추출해내어, 빈도 수에 따라 중요도를 분석하는 방법을 말합니다.

- 구성
    - word_cloud 실행함수

In [3]:
# word_cloud 실행함수

def word_cloud(lines):
    noun_list = Mecab().nouns(lines)
    df = pd.DataFrame([[k, v] for k, v in dict(Counter(noun_list)).items()], columns=["유형", "개수"])
    df = df.sort_values("개수", ascending=False).head(50)
    return {
        "fields": [
            {"name": "유형", "type": "TEXT"},
            {"name": "개수", "type": "INTEGER"}],
        "results": df.values.tolist()
    }

## word_relative / 연관어 분석
> 연관어 분석은 문서 내 추출된 단어들이 동시에 출현한 빈도를 계산해서, 키워드 간의 연관도에 따른 네트워크 구성을 보여주는 시각화 기법입니다.

- 구성
  - word_relative 실행함수
  - word_relative main logic

In [4]:
# word_relative 실행함수

def word_relative(lines):
    nlp_word_relative = NlpWordRelative()
    nlp_word_relative.lines2sentences(lines)
    df = nlp_word_relative.start()
    return {
        "fields": [
            {"name": "id", "type": "TEXT"},
            {"name": "link", "type": "TEXT"},
            {"name": "radius", "type": "INTEGER"},
            {"name": "color", "type": "TEXT"}],
        "results": df.values.tolist()
    }

In [5]:
# word_relative main logic

from colorutils import Color


class NlpWordRelative:
    def __init__(self):
        self.key_list = {}
        self.group_list = {}
        self.colors = [0, 60, 120, 240, 300]
        self.lines = None

    def create_radius_df(self, words_list, n):
        count = {}
        for line in words_list:
            words = Mecab().nouns(line)
            for i, a in enumerate(words):
                for b in words[i + 1:]:
                    if a > b:
                        count[b, a] = count.get((b, a), 0) + 1
                    else:
                        count[a, b] = count.get((a, b), 0) + 1

        count.get(("a", "b"), 0)
        df = pd.DataFrame.from_dict(count, orient='index')

        edge_list = []
        for i in range(len(df)):
            edge_list.append([df.index[i][0], df.index[i][1], df[0][i], '#bfbfbf'])

        df2 = pd.DataFrame(edge_list, columns=["id", "link", "radius", "color"])\
            .sort_values(by=['radius'], ascending=False)

        radius_df = df2.head(n)
        return radius_df

    def combine_group(self, l_idx, r_idx=None, new_group=None):
        if r_idx:
            for k, v in self.key_list.items():
                if v == r_idx:
                    self.key_list[k] = l_idx
            new_group = self.group_list.pop(r_idx)
        for k, v in new_group.items():
            if k in self.group_list[l_idx].keys():
                self.group_list[l_idx][k] = self.group_list[l_idx][k] + v
            else:
                self.group_list[l_idx][k] = v

    def key_check(self, set_id, w):
        if w[0] not in self.key_list and w[1] not in self.key_list:
            set_id += 1  # new set_id
            self.key_list[w[0]] = set_id
            self.key_list[w[1]] = set_id
            return set_id, None
        elif w[0] in self.key_list and w[1] not in self.key_list:
            self.key_list[w[1]] = self.key_list[w[0]]
            return self.key_list[w[0]], None
        elif w[1] in self.key_list and w[0] not in self.key_list:
            self.key_list[w[0]] = self.key_list[w[1]]
            return self.key_list[w[1]], None
        else:
            if self.key_list[w[0]] == self.key_list[w[1]]:
                return self.key_list[w[0]], None
            else:
                return self.key_list[w[0]], self.key_list[w[1]]

    def hsv_to_hex(self, h, s, v):
        c = Color(hsv=(h, s / 100, v / 100))
        red, green, blue = c.rgb
        ir, ig, ib = int(red), int(green), int(blue)
        return '#' + hex(ir)[2:].zfill(2) + hex(ig)[2:].zfill(2) + hex(ib)[2:].zfill(2)

    def create_vertex_list(self, nodes, hue):
        nodes_dict = dict(sorted(nodes.items(), key=lambda x: x[1], reverse=True))
        node_list = []
        s = 100
        before = 0
        for k, v in nodes_dict.items():
            if before > v:
                s -= 10
            before = v
            node_list.append([k, k, v, self.hsv_to_hex(hue, s, 100)])
        node_list_with_color = pd.DataFrame(node_list, columns=["id", "link", "radius", "color"])
        return node_list_with_color

    def manage_scale(self, nodes, max_value):
        adj_value = 20 / max_value
        for k, v in nodes.items():
            nodes[k] = v * adj_value
        return nodes

    def lines2sentences(self, lines):   # 문장의 끝맺음이 제대로 되지 않을때 문장 분리
        sentences = sent_tokenize(lines)
        for idx in range(0, len(sentences)):
            if len(sentences[idx]) <= 10:
                sentences[idx - 1] += (' ' + sentences[idx])
                sentences[idx] = ''

        encode_check_list = []
        for line in sentences:
            encode_check_list.append(line)

        self.lines = encode_check_list

    def read_file(self, path):  # 문장의 끝맺음이 제대로 될때 사용시 성능 향상
        text_file = open(path, "r")
        file_content = text_file.read()

        content_list = file_content.split("\n")
        text_file.close()

        self.lines = content_list

    def start(self):
        words_list = []
        for text in self.lines:
            words_list.append(text.strip())

        radius_df = self.create_radius_df(words_list, 100)

        i = 0
        for index, row in radius_df.iterrows():
            w1 = row['id']
            w2 = row['link']
            l_set_idx, r_set_idx = self.key_check(i, [w1, w2])
            group = {}

            if r_set_idx:
                self.combine_group(l_set_idx, r_set_idx, None)
                self.group_list[l_set_idx][w1] += row['radius']
                self.group_list[l_set_idx][w2] += row['radius']
            else:
                if l_set_idx not in self.group_list:
                    group[w1] = row['radius']
                    group[w2] = row['radius']
                    self.group_list[l_set_idx] = group
                else:
                    group[w1] = row['radius']
                    group[w2] = row['radius']
                    self.combine_group(l_set_idx, None, group)
            i += 1

        frames = []
        j = 0
        # 그룹 내 최댓값 찾기
        max_value = 0
        for g, gv in self.group_list.items():
            all_values = gv.values()
            temp = max(all_values)
            if temp > max_value:
                max_value = temp

        for k, v in self.group_list.items():
            frames.append(self.create_vertex_list(self.manage_scale(v, max_value), self.colors[j]))
            j += 1

        frames.append(radius_df)
        result = pd.concat(frames)

        return result


## key_sentence / 핵심 문장 추출
> 핵심 문장 추출을 통해 문서를 요약할 수 있는 핵심이 되는 내용을 확인할 수 있습니다.

- 구성
  - key_sentence 실행함수
  - key_sentence main logic

In [6]:
# key_sentence 실행함수

def key_sentence(lines):
    nlp_sentence = NlpSentence()
    nlp_sentence.lines2sentences(lines)
    nlp_sentence.set_summary_line_cnt(5)

    runner = nlp_sentence.run()
    return {
        "fields": [
            {"name": "핵심 문장 추출", "type": "TEXT"}],
        "results": [[res[2]] for res in runner]
    }

In [7]:
# key_sentence main logic

from konlpy.tag import Komoran
from textrank import KeysentenceSummarizer


class NlpSentence:
    def __init__(self):
        self.komoran = Komoran()
        self.sents = None
        self.cnt = 6

    def read_file(self, path):  # 문장의 끝맺음이 제대로 될때 사용시 성능 향상
        text_file = open(path, "r")
        file_content = text_file.read()

        content_list = file_content.split("\n")
        text_file.close()

        self.sents = content_list

    def lines2sentences(self, lines):   # 문장의 끝맺음이 제대로 되지 않을때 문장 분리
        sentences = sent_tokenize(lines)
        for idx in range(0, len(sentences)):
            if len(sentences[idx]) <= 10:
                sentences[idx - 1] += (' ' + sentences[idx])
                sentences[idx] = ''

        self.sents = sentences

    def komoran_tokenizer(self, sent):
        words = self.komoran.pos(sent, join=True)
        words = [w for w in words if ('/NN' in w or '/XR' in w or '/VA' in w or '/VV' in w)]
        return words

    def set_summary_line_cnt(self, cnt):
        self.cnt = cnt

    def run(self):
        summarizer = KeysentenceSummarizer(
            tokenize=self.komoran_tokenizer,
            min_sim=0.3,
            verbose=False
        )
        keysents = summarizer.summarize(self.sents, topk=self.cnt)
        return keysents


# 함수 시연
## word_cloud / 단어 빈도 분석

In [8]:
word_cloud_result = word_cloud(lines)
print(word_cloud_result)

{'fields': [{'name': '유형', 'type': 'TEXT'}, {'name': '개수', 'type': 'INTEGER'}], 'results': [['파이썬', 13], ['언어', 12], ['수', 7], ['인터프리터', 7], ['등', 5], ['이용', 5], ['속도', 5], ['프로그램', 5], ['사용', 5], ['모듈', 4], ['제한', 3], ['중요', 3], ['코드', 3], ['실행', 3], ['타이핑', 3], ['분야', 2], ['접근', 2], ['문법', 2], ['객체', 2], ['사용자', 2], ['들여쓰기', 2], ['강조', 2], ['채용', 2], ['시간', 2], ['작성', 2], ['응용', 2], ['수행', 2], ['스택', 2], ['및', 2], ['플랫폼', 2], ['프로그래밍', 2], ['기관', 2], ['동적', 2], ['소스', 1], ['때', 1], ['때문', 1], ['다음', 1], ['과학', 1], ['바이트', 1], ['공학', 1], ['컴파일', 1], ['컴퓨터', 1], ['연산', 1], ['하나', 1], ['동작', 1], ['현대', 1], ['I', 1], ['제이', 1], ['과거', 1], ['머신', 1]]}


## word_relative / 연관어 분석

In [9]:
word_relative_result = word_relative(lines)
print(word_relative_result)

{'fields': [{'name': 'id', 'type': 'TEXT'}, {'name': 'link', 'type': 'TEXT'}, {'name': 'radius', 'type': 'INTEGER'}, {'name': 'color', 'type': 'TEXT'}], 'results': [['언어', '언어', 19.999999999999996, '#ff0000'], ['등', '등', 19.740259740259738, '#ff1919'], ['속도', '속도', 19.48051948051948, '#ff3333'], ['파이썬', '파이썬', 13.506493506493506, '#ff4c4c'], ['프로그램', '프로그램', 8.83116883116883, '#ff6666'], ['수', '수', 8.571428571428571, '#ff7f7f'], ['모듈', '모듈', 5.454545454545454, '#ff9999'], ['중요', '중요', 3.3766233766233764, '#ffb2b2'], ['인터프리터', '인터프리터', 3.1168831168831166, '#ffcccc'], ['이용', '이용', 3.1168831168831166, '#ffcccc'], ['사용자', '사용자', 3.1168831168831166, '#ffcccc'], ['사용', '사용', 2.8571428571428568, '#ffe5e5'], ['분야', '분야', 2.8571428571428568, '#ffe5e5'], ['코드', '코드', 2.3376623376623376, '#ffffff'], ['응용', '응용', 2.3376623376623376, '#ffffff'], ['일반', '일반', 2.0779220779220777, '#ff118118'], ['컴퓨터', '컴퓨터', 2.0779220779220777, '#ff118118'], ['사업', '사업', 2.0779220779220777, '#ff118118'], ['생성', '생성',

## key_sentence / 핵심 문장 추출

In [10]:
key_sentence_result = key_sentence(lines)
print(key_sentence_result)

{'fields': [{'name': '핵심 문장 추출', 'type': 'TEXT'}], 'results': [['현대의 파이썬은 여전히 인터프리터 언어처럼 동작하나 사용자가 모르는 사이에 스스로 파이썬 소스 코드를 컴파일하여 바이트 코드(Byte code)를 만들어 냄으로써 다음에 수행할 때에는 빠른 속도를 보여 준다.'], ['이 문법은 파이썬에 익숙한 사용자나 기존 프로그래밍 언어에서 들여쓰기의 중요성을 높이 평가하는 사용자에게는 잘 받아들여지고 있지만, 다른 언어의 사용자에게서는 프로그래머의 코딩 스타일을 제한한다는 비판도 많다.'], ['모듈, 클래스, 객체와 같은 언어의 요소가 내부에서 접근할 수 있고, 리플렉션을 이용한 기술을 쓸 수 있다.'], ['그러나 사업 분야 등 일반적인 컴퓨터 응용 환경에서는 속도가 그리 중요하지 않고, 빠른 속도를 요하는 프로그램의 경우에도 프로토타이핑한 뒤 빠른 속도가 필요한 부분만 골라서 C 언어 등으로 모듈화할 수 있다(ctypes, SWIG, SIP 등의 래퍼 생성 프로그램들이 많이 있다).'], ['다양한 플랫폼에서 쓸 수 있고, 라이브러리(모듈)가 풍부하여, 대학을 비롯한 여러 교육 기관, 연구 기관 및 산업계에서 이용이 증가하고 있다.']]}
